Currently no picutures are readin.


In [2]:
from PyPDF2 import PdfReader

def extract_pdf_pages(pdf_path):
    reader = PdfReader(pdf_path)
    pages = []
    for i, page in enumerate(reader.pages):
        text = page.extract_text()
        pages.append({'page_number': i+1, 'text': text})
    return pages

pdf_pages = extract_pdf_pages("Makeathon TUM presentation.pdf")

pdf_pages

[{'page_number': 1, 'text': ''},
 {'page_number': 2,
  'text': 'Flow\nSpeaks2) Text Analyzer \nfor keywords1) Speech to \ntext converter3) Slide mover\nPresentation \nslide changes\nPresentation \nslide changes5) Slide \nkeyword \nclassifier4) Text \nextractor4) Image to text \nconverter\n6) Presentation \nslide number to \nkeyword tableLive Presentation app\nOffline presentation classifier'},
 {'page_number': 3,
  'text': 'Flow\nSpeaks2) Text Analyzer \nfor keywords1) Speech to \ntext converter3a) Slide \nmover\nPresentation \nslide changesLive Presentation app with slide creator\n3b) Slide creator\n3.2) Google \ninfo finder3.1) Slide \ndesign3.3) Adds slide \nto presentatio'},
 {'page_number': 4,
  'text': 'Structure of presentation\n•Introduction (ideally something funny)\n•What problem is our product solving?\n•What solutions already exist?\n•Why is our solution better?\n•How big is the business opportunity?\n•Business Model\n•Team'},
 {'page_number': 5,
  'text': 'Introduction (id

In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')  # You can pick another

for page in pdf_pages:
    if page['text']:
        page['embedding'] = model.encode(page['text'])

pdf_pages

/opt/anaconda3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

/opt/anaconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[{'page_number': 1, 'text': ''},
 {'page_number': 2,
  'text': 'Flow\nSpeaks2) Text Analyzer \nfor keywords1) Speech to \ntext converter3) Slide mover\nPresentation \nslide changes\nPresentation \nslide changes5) Slide \nkeyword \nclassifier4) Text \nextractor4) Image to text \nconverter\n6) Presentation \nslide number to \nkeyword tableLive Presentation app\nOffline presentation classifier',
  'embedding': array([-5.73790818e-02, -2.55870484e-02, -4.08065282e-02, -5.28974421e-02,
          2.35708617e-02,  3.96489799e-02,  2.23188885e-02, -2.64288336e-02,
          1.19529283e-02, -2.43925713e-02, -2.42910162e-02, -2.26396485e-03,
          9.00263805e-03,  5.52124169e-04, -1.33740706e-02, -7.50034628e-03,
         -1.34399934e-02,  8.32086205e-02,  1.19290864e-02, -6.99967816e-02,
          8.84516314e-02,  4.29103673e-02, -5.94272576e-02,  1.65329780e-02,
          4.37660627e-02,  9.81568322e-02, -5.25125526e-02,  3.43320449e-03,
          8.19259062e-02, -8.82379431e-03, -4.067683

### 4. Insert into Qdrant


In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http import models

client = QdrantClient("localhost", port=6333)

collection_name = "pdf_pages"
dim = len(pdf_pages[0]['embedding'])  # Embedding dimension

# Create the collection if not exists
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(size=dim, distance=models.Distance.COSINE),
)

# upsert the vectors
points = [
    models.PointStruct(
        id=idx,
        vector=page['embedding'],
        payload={'text': page['text'], 'page_number': page['page_number']}
    )
    for idx, page in enumerate(pdf_pages) if 'embedding' in page
]

client.upsert(
    collection_name=collection_name,
    points=points,
)

Bonus: Retrieving with Metadata
When you query Qdrant (for vector similarity), you can retrieve the payload (which includes page_number):


In [ ]:
results = client.search(
    collection_name,
    query_vector=query_embedding,
    limit=3,
    with_payload=True
)
for res in results:
    print(res.payload['page_number'], res.payload['text'])